In [1]:
import numpy as np
import pandas as pd

In [2]:
#Read in necessary data sets
#Number of students who received a certain type of discipline by district
disc = pd.read_csv(r'../Data/Florida data/FLDOE/Discipline/Discipline Concatenated.csv', index_col=0)
disc.head()

,District,Total Discipline,Year
0,ALACHUA,5791.0,2009
1,BAKER,625.0,2009
2,BAY,4970.0,2009
3,BRADFORD,1054.0,2009
4,BREVARD,10713.0,2009


In [3]:
#School grades by district
grades = pd.read_csv(r'../Data/Florida data/FLDOE/District Grades/Grades Concatenated.csv', index_col=0)
grades.head()

,District Number,District Name,Grade 2018,Grade 2017,Grade 2016,Informational Baseline Grade 2015,Grade 2014,Grade 2013,Grade 2012,Grade 2011,Grade 2010,Grade 2009,Grade 2008,Grade 2007,Grade 2006,Grade 2005,Grade 2004
0,1,ALACHUA,B,B,B,A,C,C,B,A,A,A,A,A,A,B,B
1,2,BAKER,B,B,C,B,C,C,C,B,B,B,B,C,C,C,C
2,3,BAY,B,B,B,B,C,C,B,A,B,A,A,B,A,B,B
3,4,BRADFORD,C,C,C,C,C,D,D,C,C,C,C,C,C,C,C
4,5,BREVARD,A,A,B,A,B,B,A,A,A,A,A,A,A,A,A


In [4]:
#Create new data frame in the format needed to merge all three data sets
df = pd.DataFrame(columns=['District', 'Year', 'Grade'])


In [5]:
#fill in the new data frame

for column in grades:
    for x in range(2004,2019):
        if column[-4:]== str(x):
            for index,row in grades.iterrows():
                df = df.append({'Year': x, 'District': row['District Name'], 'Grade': row[column]}, ignore_index=True)

df.head()

,District,Year,Grade
0,ALACHUA,2018,B
1,BAKER,2018,B
2,BAY,2018,B
3,BRADFORD,2018,C
4,BREVARD,2018,A


In [6]:
df['Year']=df['Year'].apply(int)

finaldf= df.merge(disc, on=["District", "Year"])
finaldf.head()

,District,Year,Grade,Total Discipline
0,HILLSBOROUGH,2017,B,29667.0
1,INDIAN RIVER,2017,B,2204.0
2,HILLSBOROUGH,2016,B,30899.0
3,INDIAN RIVER,2016,B,2386.0
4,ALACHUA,2015,A,1972.0


In [7]:
#Read District finances + juvenile crime rate merged dataset
fund = pd.read_csv(r'../Data/Florida data/merged_datasets/juvenile_crime_vs_finances.csv', index_col=0)
fund.head()

,COUNTY NAME,Year,TOTALREV,TFEDREV,TSTREV,TLOCREV,TOTALEXP,TCURINST,TCURSSVC,TCURONON,TCAPOUT,TOTAL ARRESTS,JUVENILE POP,CRIME RATE
0,Alachua,2016,9.702167,1.270125,4.333970,4.098072,9.340386,4.968435,3.352738,0.626344,0.328715,730,47667,1.531458
1,Baker,2016,8.530388,0.965414,6.065374,1.499600,8.362055,4.367453,3.266693,0.565974,0.145542,77,6877,1.119674
2,Bay,2016,9.979338,0.934416,3.981534,5.063389,9.527591,5.411936,2.667003,0.541305,0.797883,1234,39597,3.116398
3,Bradford,2016,10.011229,1.574860,5.697130,2.739239,9.352776,5.072052,3.243606,0.945415,0.083593,60,5393,1.112553
4,Brevard,2016,9.968929,1.037302,4.560299,4.371328,9.390130,5.292529,2.829184,0.455725,0.526518,2250,107226,2.098372


In [8]:
#Rename column
fund.rename(columns = {'COUNTY NAME':'District'}, inplace = True)
#Uppercase District column
fund['District'] = fund['District'].str.upper()
fund.head()


,District,Year,TOTALREV,TFEDREV,TSTREV,TLOCREV,TOTALEXP,TCURINST,TCURSSVC,TCURONON,TCAPOUT,TOTAL ARRESTS,JUVENILE POP,CRIME RATE
0,ALACHUA,2016,9.702167,1.270125,4.333970,4.098072,9.340386,4.968435,3.352738,0.626344,0.328715,730,47667,1.531458
1,BAKER,2016,8.530388,0.965414,6.065374,1.499600,8.362055,4.367453,3.266693,0.565974,0.145542,77,6877,1.119674
2,BAY,2016,9.979338,0.934416,3.981534,5.063389,9.527591,5.411936,2.667003,0.541305,0.797883,1234,39597,3.116398
3,BRADFORD,2016,10.011229,1.574860,5.697130,2.739239,9.352776,5.072052,3.243606,0.945415,0.083593,60,5393,1.112553
4,BREVARD,2016,9.968929,1.037302,4.560299,4.371328,9.390130,5.292529,2.829184,0.455725,0.526518,2250,107226,2.098372


In [9]:
Merged_DF= fund.merge(finaldf,on= ['District', 'Year'])
Merged_DF.head()

,District,Year,TOTALREV,TFEDREV,TSTREV,TLOCREV,TOTALEXP,TCURINST,TCURSSVC,TCURONON,TCAPOUT,TOTAL ARRESTS,JUVENILE POP,CRIME RATE,Grade,Total Discipline
0,HILLSBOROUGH,2016,9.901511,1.455373,4.991525,3.454613,10.253649,5.530207,2.932395,1.061683,0.544858,5800,317391,1.827399,B,30899.0
1,ALACHUA,2013,9.030259,1.249155,3.661072,4.120032,9.228024,4.746173,3.419284,0.578560,0.374721,1068,45383,2.353304,C,2458.0
2,BAKER,2013,8.153954,0.847451,5.535929,1.770574,8.243677,4.432156,3.094741,0.509233,0.178041,42,6763,0.621026,C,984.0
3,BAY,2013,8.958737,0.954382,3.050800,4.953556,8.557483,4.937448,2.346474,0.592213,0.503229,1313,37672,3.485347,C,5062.0
4,BRADFORD,2013,8.993588,1.355725,4.763664,2.874198,9.703817,5.070534,3.366107,0.941679,0.320000,52,5496,0.946143,D,920.0


In [10]:
Merged_DF.isna().sum()

District            0
Year                0
TOTALREV            0
TFEDREV             0
TSTREV              0
TLOCREV             0
TOTALEXP            0
TCURINST            0
TCURSSVC            0
TCURONON            0
TCAPOUT             0
TOTAL ARRESTS       0
JUVENILE POP        0
CRIME RATE          0
Grade               0
Total Discipline    0
dtype: int64

In [11]:
#transform data set since sklearn does not accept features in categorical data

Merged_DF = Merged_DF.replace({'Grade': {'A': 1, 'B': 2, 'C':3, 'D':4, 'F':5}})
Merged_DF.head()

,District,Year,TOTALREV,TFEDREV,TSTREV,TLOCREV,TOTALEXP,TCURINST,TCURSSVC,TCURONON,TCAPOUT,TOTAL ARRESTS,JUVENILE POP,CRIME RATE,Grade,Total Discipline
0,HILLSBOROUGH,2016,9.901511,1.455373,4.991525,3.454613,10.253649,5.530207,2.932395,1.061683,0.544858,5800,317391,1.827399,2,30899.0
1,ALACHUA,2013,9.030259,1.249155,3.661072,4.120032,9.228024,4.746173,3.419284,0.578560,0.374721,1068,45383,2.353304,3,2458.0
2,BAKER,2013,8.153954,0.847451,5.535929,1.770574,8.243677,4.432156,3.094741,0.509233,0.178041,42,6763,0.621026,3,984.0
3,BAY,2013,8.958737,0.954382,3.050800,4.953556,8.557483,4.937448,2.346474,0.592213,0.503229,1313,37672,3.485347,3,5062.0
4,BRADFORD,2013,8.993588,1.355725,4.763664,2.874198,9.703817,5.070534,3.366107,0.941679,0.320000,52,5496,0.946143,4,920.0


In [12]:
list(Merged_DF)

['District',
 'Year',
 'TOTALREV',
 'TFEDREV',
 'TSTREV',
 'TLOCREV',
 'TOTALEXP',
 'TCURINST',
 'TCURSSVC',
 'TCURONON',
 'TCAPOUT',
 'TOTAL ARRESTS',
 'JUVENILE POP',
 'CRIME RATE',
 'Grade',
 'Total Discipline']

In [49]:
cols= [
 'CRIME RATE',
 'Grade',
'Total Discipline']

features = Merged_DF[cols]

features.head()



,CRIME RATE,Grade,Total Discipline
0,1.827399,2,30899.0
1,2.353304,3,2458.0
2,0.621026,3,984.0
3,3.485347,3,5062.0
4,0.946143,4,920.0


In [43]:
target = Merged_DF['TOTALREV']
target.head()

0    9.901511
1    9.030259
2    8.153954
3    8.958737
4    8.993588
Name: TOTALREV, dtype: float64

In [44]:
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)

In [45]:
#Create a decision Tree and calculate the score

regressor = DecisionTreeRegressor(random_state=0, min_samples_leaf=10, max_depth=3)
model = regressor.fit(X_train, y_train)
model.score(X_test, y_test)

0.14911541778507476

In [46]:
import graphviz 
from sklearn import tree

dot_data = tree.export_graphviz(model, out_file=None, feature_names=list(X_train)) 

graph = graphviz.Source(dot_data) 

graph.render("disc_grades_tree1") 

'disc_grades_tree1.pdf'

In [47]:
import statsmodels.api as sm

model = sm.OLS(y_train, X_train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               TOTALREV   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     735.0
Date:                Sun, 25 Nov 2018   Prob (F-statistic):          3.05e-131
Time:                        20:48:17   Log-Likelihood:                -727.69
No. Observations:                 279   AIC:                             1461.
Df Residuals:                     276   BIC:                             1472.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
CRIME RATE           1.9277      0.161     11.937      0.000       1.610       2.246
Grade                2.2905      0.137     16.702      0.000       2.021       2.560
Total Discipline  4.025e-05   2.26e-05      1.782      0.076   -4.22e-06    8.47e-05
==============================================================================
Omnibus:                        1.745   Durbin-Watson:                   1.813
Prob(Omnibus):                  0.418   Jarque-Bera (JB):                1.514
Skew:                           0.026   Prob(JB):                        0.469
Kurtosis:                       2.643   Cond. No.                     1.08e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.08e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [48]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)

mean_squared_error(y_test, y_pred)

12.639843308826247